# **Homework 2-1 Phoneme Classification**

## The DARPA TIMIT Acoustic-Phonetic Continuous Speech Corpus (TIMIT)
The TIMIT corpus of reading speech has been designed to provide speech data for the acquisition of acoustic-phonetic knowledge and for the development and evaluation of automatic speech recognition systems.

This homework is a multiclass classification task, 
we are going to train a deep neural network classifier to predict the phonemes for each frame from the speech corpus TIMIT.

link: https://academictorrents.com/details/34e2b78745138186976cbc27939b1b34d18bd5b3

## Download Data
Download data from google drive, then unzip it.

You should have `timit_11/train_11.npy`, `timit_11/train_label_11.npy`, and `timit_11/test_11.npy` after running this block.<br><br>
`timit_11/`
- `train_11.npy`: training data<br>
- `train_label_11.npy`: training label<br>
- `test_11.npy`:  testing data<br><br>

**notes: if the google drive link is dead, you can download the data directly from Kaggle and upload it to the workspace**




In [1]:
#!gdown --id '1HPkcmQmFGu-3OknddKIa5dNDsR05lIQR' --output data.zip
#!unzip data.zip
#!ls 

## Preparing Data
Load the training and testing data from the `.npy` file (NumPy array).

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np

print('Loading data ...')

data_root='/content/drive/MyDrive/timit_11/'
train = np.load(data_root + 'train_11.npy')
train_label = np.load(data_root + 'train_label_11.npy')
test = np.load(data_root + 'test_11.npy')

print('Size of training data: {}'.format(train.shape))
print('Size of testing data: {}'.format(test.shape))

Loading data ...
Size of training data: (1229932, 429)
Size of testing data: (451552, 429)


In [4]:
import torch
from torch.utils.data import Dataset

class TIMITDataset(Dataset):
    def __init__(self, X, y=None):
        self.data = torch.from_numpy(X).float()
        if y is not None:
            y = y.astype(np.int)
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]

    def __len__(self):
        return len(self.data)


Split the labeled data into a training set and a validation set, you can modify the variable `VAL_RATIO` to change the ratio of validation data.

In [5]:
VAL_RATIO = 0.02

percent = int(train.shape[0] * (1 - VAL_RATIO))
train_x, train_y, val_x, val_y = train[:percent], train_label[:percent], train[percent:], train_label[percent:]
print('Size of training set: {}'.format(train_x.shape))
print('Size of validation set: {}'.format(val_x.shape))

Size of training set: (1205333, 429)
Size of validation set: (24599, 429)


Create a data loader from the dataset, feel free to tweak the variable `BATCH_SIZE` here.

In [6]:
BATCH_SIZE = 32

from torch.utils.data import DataLoader

train_set = TIMITDataset(train_x, train_y)
val_set = TIMITDataset(val_x, val_y)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True) #only shuffle the training data
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False)

Cleanup the unneeded variables to save memory.<br>

**notes: if you need to use these variables later, then you may remove this block or clean up unneeded variables later<br>the data size is quite huge, so be aware of memory usage in colab**

In [7]:
import gc

del train, train_label, train_x, train_y, val_x, val_y
gc.collect()

153

## Create Model

Define model architecture, you are encouraged to change and experiment with the model architecture.

In [8]:
import torch
import torch.nn as nn

class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.layer1 = nn.Linear(429, 2048)
        self.layer2 = nn.Linear(2048, 1024)
        self.layer3 = nn.Linear(1024, 512)
        self.layer4 = nn.Linear(512, 256)
        self.layer5 = nn.Linear(256, 128)
        self.out = nn.Linear(128, 39)

        """
        self.layer1 = nn.Linear(429, 1024)
        self.layer2 = nn.Linear(1024, 512)
        self.layer3 = nn.Linear(512, 128)
        self.out = nn.Linear(128, 39) 
        """

        self.act_fn = nn.ReLU()
        self.BN0 = nn.BatchNorm1d(429)
        self.BN1 = nn.BatchNorm1d(2048)
        self.BN2 = nn.BatchNorm1d(1024)
        self.BN3 = nn.BatchNorm1d(512)
        self.BN4 = nn.BatchNorm1d(256)
        self.BN5 = nn.BatchNorm1d(128)
        self.drop_out1 = nn.Dropout(0.3)
        self.drop_out2 = nn.Dropout(0.5)

    def forward(self, x):
        x = self.BN0(x)

        x = self.layer1(x)
        x = self.BN1(x)
        x = self.act_fn(x)
        x = self.drop_out1(x)

        x = self.layer2(x)
        x = self.BN2(x)
        x = self.act_fn(x)
        x = self.drop_out1(x)

        x = self.layer3(x)
        x = self.BN3(x)
        x = self.act_fn(x)
        x = self.drop_out1(x)

        x = self.layer4(x)
        x = self.BN4(x)
        x = self.act_fn(x)
        x = self.drop_out1(x)

        x = self.layer5(x)
        x = self.BN5(x)
        x = self.act_fn(x)
        x = self.drop_out1(x)

        x = self.out(x)
        
        return x

## Training

In [9]:
#check device
def get_device():
  return 'cuda' if torch.cuda.is_available() else 'cpu'

Fix random seeds for reproducibility.

In [10]:
# fix random seed
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

Feel free to change the training parameters here.

In [20]:
# fix random seed for reproducibility
same_seeds(0)

# get device 
device = get_device()
print(f'DEVICE: {device}')

# training parameters
num_epoch = 100               # number of training epoch
learning_rate = 0.0001       # learning rate

# the path where checkpoint saved
model_saved_path = '/content/drive/MyDrive/model/model.ckpt'
model_path = './model.ckpt'

# create model, define a loss function, and optimizer
model = Classifier().to(device)
criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=0.0001)

# checkpoint mode
checkpoint = torch.load(model_path)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']


DEVICE: cuda


In [21]:
# start training
best_acc = checkpoint['best_acc']
#best_acc = 0.0
early_stop_cnt = 0
for epoch in range(num_epoch):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0

    # training
    model.train() # set the model to training mode
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad() 
        outputs = model(inputs) 
        batch_loss = criterion(outputs, labels)
        _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        batch_loss.backward() 
        optimizer.step() 

        train_acc += (train_pred.cpu() == labels.cpu()).sum().item()
        train_loss += batch_loss.item()

    # validation
    if len(val_set) > 0:
        model.eval() # set the model to evaluation mode
        with torch.no_grad():
            for i, data in enumerate(val_loader):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                batch_loss = criterion(outputs, labels) 
                _, val_pred = torch.max(outputs, 1) 
            
                val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
                val_loss += batch_loss.item()

            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
                epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader), val_acc/len(val_set), val_loss/len(val_loader)
            ))

            # if the model improves, save a checkpoint at this epoch
            if val_acc > best_acc:
                best_acc = val_acc
                torch.save(model.state_dict(), model_path)
                torch.save({
                              'epoch': epoch,
                              'model_state_dict': model.state_dict(),
                              'optimizer_state_dict': optimizer.state_dict(),
                              'train_loss': train_loss,
                              'train_acc': train_acc,
                              'val_loss': val_loss,
                              'val_acc': val_acc,
                              'best_acc': best_acc 
                            }, model_path)
                early_stop_cnt = 0
                print('saving model with acc {:.3f}'.format(best_acc/len(val_set)))

    else:
        print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
            epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader)
        ))
  

# if not validating, save the last epoch
if len(val_set) == 0:
    torch.save(model.state_dict(), model_path)
    print('saving model at last epoch')


[001/063] Train Acc: 0.713584 Loss: 0.906203 | Val Acc: 0.749705 loss: 0.754961
[002/063] Train Acc: 0.715395 Loss: 0.899150 | Val Acc: 0.752632 loss: 0.749748
saving model with acc 0.753
[003/063] Train Acc: 0.717286 Loss: 0.891830 | Val Acc: 0.748282 loss: 0.758094
[004/063] Train Acc: 0.718684 Loss: 0.884890 | Val Acc: 0.748770 loss: 0.765342
[005/063] Train Acc: 0.720156 Loss: 0.878396 | Val Acc: 0.750437 loss: 0.749358
[006/063] Train Acc: 0.722571 Loss: 0.873672 | Val Acc: 0.750559 loss: 0.754076
[007/063] Train Acc: 0.723427 Loss: 0.867849 | Val Acc: 0.750437 loss: 0.747407
[008/063] Train Acc: 0.725297 Loss: 0.862569 | Val Acc: 0.755722 loss: 0.741597
saving model with acc 0.756
[009/063] Train Acc: 0.725940 Loss: 0.857677 | Val Acc: 0.755478 loss: 0.743713
[010/063] Train Acc: 0.727770 Loss: 0.853112 | Val Acc: 0.755234 loss: 0.740172
[011/063] Train Acc: 0.728726 Loss: 0.849264 | Val Acc: 0.754177 loss: 0.744440
[012/063] Train Acc: 0.729637 Loss: 0.845718 | Val Acc: 0.753486

## Testing

Create a testing dataset, and load model from the saved checkpoint.

In [13]:
# create testing dataset
test_set = TIMITDataset(test, None)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

# create model and load weights from checkpoint
model = Classifier().to(device)
checkpoint = torch.load(model_path)
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

Make prediction.

In [14]:
predict = []
model.eval() # set the model to evaluation mode
with torch.no_grad():
    for i, data in enumerate(test_loader):
        inputs = data
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability

        for y in test_pred.cpu().numpy():
            predict.append(y)

Write prediction to a CSV file.

After finish running this block, download the file `prediction.csv` from the files section on the left-hand side and submit it to Kaggle.

In [15]:
with open('prediction.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(predict):
        f.write('{},{}\n'.format(i, y))

In [17]:
torch.save(model.state_dict(), "/content/drive/MyDrive/tmp_model/model.ckpt")
torch.save({
                              'epoch': epoch,
                              'model_state_dict': model.state_dict(),
                              'optimizer_state_dict': optimizer.state_dict(),
                              'train_loss': train_loss,
                              'train_acc': train_acc,
                              'val_loss': val_loss,
                              'val_acc': val_acc,
                              'best_acc': best_acc 
                            }, "/content/drive/MyDrive/tmp_model/model.ckpt")